In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import sklearn
import warnings
warnings.filterwarnings("ignore")
import scipy
import sys
import time

In [2]:
Start_Time = time.time()

##### Defining parameters

In [4]:
No_of_Iteration = 10000
No_of_Components = 3
Levels = 2
Recovery_Cutoff_List = np.array([0.5 + (np.random.randint(1,5)/10) for i in range(No_of_Components)])
Lamda_list = np.array([1/400,2/400,1/600,2/600,1/800,2/800])

In [5]:
Recovery_Cutoff_List

array([0.9, 0.7, 0.8])

##### Random Numbers and Failure Time of the Components

In [ ]:
random_table = pd.DataFrame(np.zeros((No_of_Components*Levels,1))).T
failure_table = pd.DataFrame(np.zeros((No_of_Components*Levels,1))).T
for i in range(No_of_Iteration):
    random = pd.Series([np.round(np.random.random(),4)for k in range(No_of_Components*Levels)])
    random_table.loc[i] = random
    for j in range(No_of_Components*Levels):
        failure_table.loc[i,j] = np.round(-1*np.log(random_table.iloc[i,j])/Lamda_list[j],1)

In [ ]:
random_table.head(2)

In [ ]:
failure_table.head(2)

##### Random Numbers to pick the New or Remanufacturing one.

In [ ]:
Pick_Table = pd.DataFrame(np.zeros((No_of_Components,1))).T
for i in range(No_of_Iteration):
    random = ([np.round(np.random.random(),1)for k in range(No_of_Components)])
    Pick_Table.loc[i] = random
Pick_Table.head(2)

##### Choosing the Right Level based on generated random number and cutoff for each component

In [ ]:
failure_data_table_after_pickup = pd.DataFrame(np.zeros((No_of_Components,1))).T
for i in range(No_of_Iteration):
    for j in range(No_of_Components):
        if Pick_Table.iloc[i,j] >= Recovery_Cutoff_List[j]:
            failure_data_table_after_pickup.loc[i,j] = failure_table.iloc[i,2*j]
        else:
            failure_data_table_after_pickup.loc[i,j] = failure_table.iloc[i,2*j+1]
failure_data_table_after_pickup.head(2)

##### Product Reliability Assuming All Components are in Series

In [ ]:
data = pd.DataFrame()
for i in range(No_of_Components):
    data['failure_time_of_component'+str(i+1)] = failure_data_table_after_pickup.iloc[:,i]
system_failure_time = pd.Series([min(data.iloc[j,:]) for j in range(No_of_Iteration)])
data['system_failure_time'] = system_failure_time
for i in range(No_of_Components):
    data['recovery_cutoff_for_component'+str(i+1)] = Recovery_Cutoff_List[i]

In [ ]:
data.head(2)

In [ ]:
file_name = str(No_of_Components)+'_components_series_system_failure_data_assuming_expo_dist_with_cutoff_'+str(Recovery_Cutoff_List)+'.csv'
data.to_csv(file_name)

In [ ]:
End_time = time.time()
print(np.round(End_time-Start_Time,2))